# Download YARA Retro matches - A1000
This notebook offers the option of downloading multiple files matched by YARA Retroactive hunt.  
To create YARA rulesets and start YARA Retroactive hunt, use other Python methods from the ReversingLabs SDK.


### Used A1000 functions
- **get_yara_ruleset_matches_v2**
- **download_sample**

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

In [ ]:
import json
import os
from ReversingLabs.SDK.a1000 import A1000


CREDENTIALS = json.load(open('credentials.json'))
HOST = CREDENTIALS.get("a1000").get("a1000_url")
TOKEN = CREDENTIALS.get("a1000").get("token")

cwd = os.getcwd()
upper_level = os.path.dirname(cwd)
USER_AGENT = json.load(open(os.path.join(upper_level, "user_agent.json")))["user_agent"]

a1000 = A1000(
    host=HOST,
    token=TOKEN,
    user_agent=USER_AGENT,
    verify=False
)


### 1. Batch-download function
This function will be used for downloading each batch of matched samples. Execute it so you can use it further on in the notebook.

In [ ]:

def download_batch(sample_list, download_path):
    for sample in sample_list:
        response = a1000.download_sample(sample_hash=sample.get("sha1"))
        
        with open(os.path.join(download_path, sample.get("sha1")), "wb") as file_handle:
            file_handle.write(response.content)
            

### 2. Main function
To retrieve batches of samples matched by a YARA Retro ruleset and download them as files, use the `download_retro_hunt_matches` function.  
This function accepts the following parameters:
- `ruleset_name`: the name of the ruleset whose matches you want to download as files
- `download_path`: needs to be a full path to the existing folder
- `max_results`: the maximum number of matches you want retrieved and downloaded
- `records_per_page`: matches are retrieved in pages. this number defines how big each page will be

Execute the function so you can use it multiple times.


In [ ]:
def download_retro_hunt_matches(ruleset_name, download_path, records_per_page=500, max_results=None):    
    page_number = 1
    more_pages = True
    result_count = 0
    
    while more_pages:        
        response = a1000.get_yara_ruleset_matches_v2(
            ruleset_name=ruleset_name,
            page_size=records_per_page,
            page=page_number
        )
        
        resp_json = response.json()
        results = resp_json.get("results", [])
        more_pages = resp_json.get("next")
        
        result_count += len(results)
        page_number += 1
        
        if max_results:
                        
            if result_count >= max_results:
                excess = result_count - max_results            
                cutoff = len(results) - excess
                
                results = results[:cutoff]
                download_batch(sample_list=results, download_path=download_path)
                
                break
        
        download_batch(sample_list=results, download_path=download_path)
        
        if not more_pages:
            break
     

### 3. Run

In [ ]:
download_retro_hunt_matches(
    ruleset_name="ChangeMe",
    download_path="/change/me",
    records_per_page=20,
    max_results=100
)